In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-nlp-lab1-problem2/submission_example.csv
/kaggle/input/2023-nlp-lab1-problem2/simple_seq.train.csv
/kaggle/input/2023-nlp-lab1-problem2/simple_seq.test.csv


In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Bidirectional, CuDNNGRU, Activation, CuDNNLSTM 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.metrics import MeanSquaredError
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
#from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dropout
from scipy import interpolate
from scipy import stats
import tensorflow as tf
from tensorflow.keras.initializers import GlorotUniform






with open("/kaggle/input/2023-nlp-lab1-problem2/simple_seq.train.csv") as f:
    lines = f.read() ##Assume the sample file has 3 lines
    lines = "".join([s for s in lines.strip().splitlines(True) if s.strip()])
    first = lines.split('\n', -1)
    train_list = []
    for i in range(len(first)):
        strings = first[i].replace(" ","")
        strings = first[i].split(',', -1)
        strings = [v for v in strings if v]         # 빈요소 "" 제거 
        strings = " ".join(strings)
        train_list.append(strings)
        
        
train_list[0:10]

['W25 W26 W27 W19 W28 W29 W30 W31 W32 W33 W34 W35 W36 W37 W38 W39 W24 W40 D11',
 'W41 W4 W42 W43 W44 W45 W46 W47 W48 W49 W50 W51 W52 W53 W17 W54 W24 D1',
 'W55 W19 W46 W32 W32 W56 W57 W58 W59 W19 W13 W60 W19 W13 W61 W62 D3',
 'W13 W83 W32 W32 W56 W57 W13 W84 W19 W28 W85 W86 W24 D20',
 'W87 W88 W89 W90 W32 W91 W13 W92 W93 W90 W94 W95 W24 D20',
 'W13 W52 W32 W53 W17 W13 W96 W97 W10 W2 W98 W99 W19 W13 W100 W24 D20',
 'W122 W123 W110 W124 W125 W19 W13 W126 W127 W128 W32 W129 W130 W36 W13 W131 W132 W17 W133 W24 D20',
 'W1 W2 W3 W4 W134 W7 W28 W78 W19 W13 W135 W136 W137 W138 W139 W90 W17 W140 W24 D1',
 'W154 W155 W134 W47 W139 W28 W156 W157 W97 W90 W158 W159 W46 W32 W154 W46 W160 W161 W24 D20',
 'W17 W162 W12 W163 W144 W12 W55 W164 W165 W32 W166 W167 W46 W168 W32 W169 W24 D20']

In [3]:
words = []       # train_words
for s in train_list:     # 문장에서 단어 단위로 쪼개기 
    words.extend(s.split())
words[0:40], len(words)

(['W25',
  'W26',
  'W27',
  'W19',
  'W28',
  'W29',
  'W30',
  'W31',
  'W32',
  'W33',
  'W34',
  'W35',
  'W36',
  'W37',
  'W38',
  'W39',
  'W24',
  'W40',
  'D11',
  'W41',
  'W4',
  'W42',
  'W43',
  'W44',
  'W45',
  'W46',
  'W47',
  'W48',
  'W49',
  'W50',
  'W51',
  'W52',
  'W53',
  'W17',
  'W54',
  'W24',
  'D1',
  'W55',
  'W19',
  'W46'],
 12335)

# Word to id

In [4]:
##print("words : ", words)

# 중복단어 제거
words = list(dict.fromkeys(words))

# 각 단어별 일련번호
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    word_to_id[w] = len(word_to_id)

# 각 번호별 단어
id_to_word = {i: w for w, i in word_to_id.items()}

word_to_id, len(word_to_id)

({'[PAD]': 0,
  '[UNK]': 1,
  'W25': 2,
  'W26': 3,
  'W27': 4,
  'W19': 5,
  'W28': 6,
  'W29': 7,
  'W30': 8,
  'W31': 9,
  'W32': 10,
  'W33': 11,
  'W34': 12,
  'W35': 13,
  'W36': 14,
  'W37': 15,
  'W38': 16,
  'W39': 17,
  'W24': 18,
  'W40': 19,
  'D11': 20,
  'W41': 21,
  'W4': 22,
  'W42': 23,
  'W43': 24,
  'W44': 25,
  'W45': 26,
  'W46': 27,
  'W47': 28,
  'W48': 29,
  'W49': 30,
  'W50': 31,
  'W51': 32,
  'W52': 33,
  'W53': 34,
  'W17': 35,
  'W54': 36,
  'D1': 37,
  'W55': 38,
  'W56': 39,
  'W57': 40,
  'W58': 41,
  'W59': 42,
  'W13': 43,
  'W60': 44,
  'W61': 45,
  'W62': 46,
  'D3': 47,
  'W83': 48,
  'W84': 49,
  'W85': 50,
  'W86': 51,
  'D20': 52,
  'W87': 53,
  'W88': 54,
  'W89': 55,
  'W90': 56,
  'W91': 57,
  'W92': 58,
  'W93': 59,
  'W94': 60,
  'W95': 61,
  'W96': 62,
  'W97': 63,
  'W10': 64,
  'W2': 65,
  'W98': 66,
  'W99': 67,
  'W100': 68,
  'W122': 69,
  'W123': 70,
  'W110': 71,
  'W124': 72,
  'W125': 73,
  'W126': 74,
  'W127': 75,
  'W128': 76,


# Divide the Train_data and Label

In [5]:
#########################   TRAIN   ################################

with open("/kaggle/input/2023-nlp-lab1-problem2/simple_seq.train.csv") as f:
    lines = f.read()
    lines = "".join([s for s in lines.strip().splitlines(True) if s.strip()])
    first = lines.split('\n', -1)
    data_list = []
    label_list = []
    for i in range(len(first)):
        strings = first[i].replace(" ","")
        strings = first[i].split(',', -1)
        strings = [v for v in strings if v]         # 빈요소 "" 제거 
        label_list.append(strings.pop())
        strings = " ".join(strings)
        data_list.append(strings)
        
print("data[0:20] : \n", data_list[0:20], len(data_list)) 
print("\nlabel[0:20] : \n",label_list[0:20], len(label_list))

data[0:20] : 
 ['W25 W26 W27 W19 W28 W29 W30 W31 W32 W33 W34 W35 W36 W37 W38 W39 W24 W40', 'W41 W4 W42 W43 W44 W45 W46 W47 W48 W49 W50 W51 W52 W53 W17 W54 W24', 'W55 W19 W46 W32 W32 W56 W57 W58 W59 W19 W13 W60 W19 W13 W61 W62', 'W13 W83 W32 W32 W56 W57 W13 W84 W19 W28 W85 W86 W24', 'W87 W88 W89 W90 W32 W91 W13 W92 W93 W90 W94 W95 W24', 'W13 W52 W32 W53 W17 W13 W96 W97 W10 W2 W98 W99 W19 W13 W100 W24', 'W122 W123 W110 W124 W125 W19 W13 W126 W127 W128 W32 W129 W130 W36 W13 W131 W132 W17 W133 W24', 'W1 W2 W3 W4 W134 W7 W28 W78 W19 W13 W135 W136 W137 W138 W139 W90 W17 W140 W24', 'W154 W155 W134 W47 W139 W28 W156 W157 W97 W90 W158 W159 W46 W32 W154 W46 W160 W161 W24', 'W17 W162 W12 W163 W144 W12 W55 W164 W165 W32 W166 W167 W46 W168 W32 W169 W24', 'W134 W7 W28 W201 W12 W202 W203 W12 W204 W205 W24', 'W46 W206 W7 W8 W207 W208 W12 W28 W209 W210 W19 W28 W211 W19 W13 W212 W213 W17 W205 W24', 'W90 W214 W32 W206 W7 W17 W13 W215 W24', 'W55 W131 W212 W216 W47 W32 W7 W17 W13 W217 W218 W97 W28 W219 W24

# Integer encoding and Padding

In [6]:
# 일련번호 데이터
train_inputs = []
for s in data_list:                                 ##### 문장별 반복 
    row = [word_to_id[w] for w in s.split()]
##    print(row)                                    ##### 번호 나열
    row += [0] * (20 - len(row))                    ##### padding, later we use pad id in dictionary
    train_inputs.append(row)
train_inputs = np.array(train_inputs)

print("data[0:20] : \n", train_inputs, train_inputs.shape)

data[0:20] : 
 [[   2    3    4 ...   19    0    0]
 [  21   22   23 ...    0    0    0]
 [  38    5   27 ...    0    0    0]
 ...
 [ 199  737 1415 ...    0    0    0]
 [ 199  328 2563 ...    0    0    0]
 [ 199   28   16 ...    0    0    0]] (900, 20)


In [7]:
label_inputs = []
for s in label_list:
    row = [word_to_id[w] for w in s.split()]
    label_inputs.append(row)
label_inputs = np.array(label_inputs)
print("label[0:20] : \n", label_inputs[0:20], label_inputs.shape)

label[0:20] : 
 [[ 20]
 [ 37]
 [ 47]
 [ 52]
 [ 52]
 [ 52]
 [ 52]
 [ 37]
 [ 52]
 [ 52]
 [ 52]
 [125]
 [ 52]
 [132]
 [132]
 [147]
 [ 52]
 [ 52]
 [164]
 [176]] (900, 1)


# Load_Test_set

In [8]:
###########################   TEST_SET   ######################################
with open("/kaggle/input/2023-nlp-lab1-problem2/simple_seq.test.csv") as f:
    lines = f.read() ##Assume the sample file has 3 lines
    lines = "".join([s for s in lines.strip().splitlines(True) if s.strip()])
    first = lines.split('\n', -1)
    test_list = []
    for i in range(len(first)):
        strings = first[i].replace(" ","")
        strings = first[i].split(',', -1)
        strings = [v for v in strings if v]         # 빈요소 "" 제거 
        strings = " ".join(strings)
        test_list.append(strings)
        
test_list[0:10]

['W13 W81 W19 W346 W846 W1582 W70 W28 W5433 W19 W1163 W2261 W24',
 'W5413 W111 W5414 W32 W68 W5415 W12 W2402 W19 W5438 W5439 W5440 W12 W346 W240 W5441 W5442 W24',
 'W5413 W111 W5414 W32 W68 W5415 W12 W417 W346 W336 W17 W28 W5443 W12 W122 W47 W38 W335 W1248 W24',
 'W5413 W111 W5414 W32 W68 W5415 W12 W346 W32 W2833 W93 W28 W5444 W5445 W17 W346 W5446 W24',
 'W5413 W111 W5414 W32 W68 W5415 W12 W111 W346 W47 W336 W286 W5415 W552 W5447 W641 W346 W24',
 'W5413 W111 W5414 W32 W68 W5415 W12 W346 W168 W2464 W5448 W24',
 'W87 W31 W47 W38 W1196 W97 W627 W5449',
 'W447 W28 W498 W204 W2590 W5451 W24',
 'W475 W32 W246 W337 W1254 W1198 W461',
 'W110 W904 W90 W110 W904 W87 W38 W498 W90 W81 W97 W15 W1065 W555 W475 W90 W47 W38 W280 W24']

In [9]:
test_inputs = []
for s in test_list:
    row = []
    for w in s.split():
        if w in word_to_id:
            row.append(word_to_id[w])
        else:
            row.append(word_to_id['[UNK]'])  # 'W846' 대신 'UNK'를 사용
    row += [0] * (20 - len(row))
    test_inputs.append(row)
test_inputs = np.array(test_inputs)

test_inputs, test_inputs.shape

(array([[  43,  802,    5, ...,    0,    0,    0],
        [2537,  328, 2538, ...,   18,    0,    0],
        [2537,  328, 2538, ...,  188,  727,   18],
        ...,
        [ 205,  355,   59, ...,    0,    0,    0],
        [  55,   53,  403, ...,    0,    0,    0],
        [   1,  102,   71, ...,    0,    0,    0]]),
 (100, 20))

In [10]:
vocab_size = len(word_to_id)
EMBEDDING_SIZE = 1024
embeddings = np.random.randn(vocab_size, EMBEDDING_SIZE)
embeddings, embeddings.shape 

(array([[ 0.08181538, -1.11618727,  0.3949702 , ...,  0.07243768,
          0.91730544,  0.21661305],
        [-1.13637254, -0.37439796, -1.10473781, ..., -1.069971  ,
         -2.0059688 ,  1.79621251],
        [ 0.74986233,  0.13079743,  0.60353269, ..., -0.96900658,
         -1.78592776, -1.92081582],
        ...,
        [-0.13741363, -0.76402701,  0.85756657, ..., -1.55858277,
         -0.45858667,  0.37200918],
        [ 2.01012851, -1.22775891, -0.44469232, ..., -0.25242779,
         -0.09001727,  0.37168003],
        [-1.07296078, -0.56730458, -0.81276934, ...,  0.75487275,
         -0.9124351 , -0.22944681]]),
 (2567, 1024))

In [11]:
train = embeddings[train_inputs]
train = train.reshape(train.shape[0],train.shape[1]*train.shape[2])
X_train = train
X_train, X_train.shape

(array([[ 0.74986233,  0.13079743,  0.60353269, ...,  0.07243768,
          0.91730544,  0.21661305],
        [ 0.60438267, -1.39879817,  0.85212724, ...,  0.07243768,
          0.91730544,  0.21661305],
        [ 2.41067104, -0.64734006,  0.46702607, ...,  0.07243768,
          0.91730544,  0.21661305],
        ...,
        [-0.81876007,  0.87634017,  1.23396578, ...,  0.07243768,
          0.91730544,  0.21661305],
        [-0.81876007,  0.87634017,  1.23396578, ...,  0.07243768,
          0.91730544,  0.21661305],
        [-0.81876007,  0.87634017,  1.23396578, ...,  0.07243768,
          0.91730544,  0.21661305]]),
 (900, 20480))

In [12]:
onehot_metrix = np.eye(len(word_to_id))
onehot_metrix, onehot_metrix.shape
label_onehots = onehot_metrix[label_inputs]
flatten_label_onehots= label_onehots.reshape(label_onehots.shape[0],label_onehots.shape[2])
label_onehots.shape, flatten_label_onehots, flatten_label_onehots.shape
y_train = flatten_label_onehots

In [13]:
X_test = embeddings[test_inputs]
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
X_test[0:20], X_test.shape

(array([[-0.28961505, -1.33766027,  0.29409028, ...,  0.07243768,
          0.91730544,  0.21661305],
        [-1.68114221,  0.3385586 ,  0.0913771 , ...,  0.07243768,
          0.91730544,  0.21661305],
        [-1.68114221,  0.3385586 ,  0.0913771 , ...,  0.14758096,
          0.33133842, -0.82885428],
        ...,
        [-0.02815954,  0.60704294,  3.02949846, ...,  0.07243768,
          0.91730544,  0.21661305],
        [ 1.5922265 ,  0.65735952, -0.93833681, ...,  0.07243768,
          0.91730544,  0.21661305],
        [ 2.89652332,  0.86793524,  0.08694349, ...,  0.07243768,
          0.91730544,  0.21661305]]),
 (100, 20480))

In [14]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Layer

class DenseLayer(Layer):
    def __init__(self, units, activation='sigmoid', kernel_initializer='glorot_uniform', **kwargs):
        super(DenseLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.kernel_initializer = kernel_initializer

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=(input_shape[-1], self.units), initializer=self.kernel_initializer, trainable=True)
        self.bias = self.add_weight(name='bias', shape=(self.units,), initializer='zeros', trainable=True)
        super(DenseLayer, self).build(input_shape)

    def call(self, inputs):
        outputs = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            activation_layer = Activation(self.activation)
            outputs = activation_layer(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)
def DeepNN(X_train, y_train, X_test):# ,sc):
    # create a model 
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Embedding, Flatten, BatchNormalization

    learning_rate = 0.001
    weight_decay = 0.0001
    hidden_layer_1 = 1000
    hidden_layer_2 = 100
    input_dim = X_train.shape[1]
    output_dim = y_train.shape[1]
    

    model = Sequential()
    model.add(DenseLayer(hidden_layer_1, activation='sigmoid', input_dim = X_train.shape[1], kernel_initializer=GlorotUniform()))
    model.add(BatchNormalization())
    model.add(DenseLayer(hidden_layer_2, activation='sigmoid', kernel_initializer=GlorotUniform()))
    model.add(BatchNormalization())
    model.add(DenseLayer(output_dim, activation='softmax'))

    


    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=weight_decay, nesterov=True),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    hist_gru = model.fit(X_train, y_train, epochs=200, validation_split=0.1, batch_size=64)
          
    prediction = model.predict(X_test)

    
    return model, prediction, hist_gru


model, prediction, hist_gru = DeepNN(X_train, y_train, X_test)#, sc)


#print("train_onehots.shape, test_onehots.shape : ", train_onehots.shape, test_onehots.shape)
#print("flatten_train_onehots.shape, flatten_test_onehots.shape : ", flatten_train_onehots.shape, flatten_test_onehots.shape)

predicted_labels = np.argmax(prediction, axis=1)

print(prediction,prediction.shape)

pred_label_list = []
for label in predicted_labels:
    pred_label_list.append(id_to_word[label])
    print(id_to_word[label])

Epoch 1/200
13/13 [==============================] - 5s 306ms/step - loss: 7.8704 - accuracy: 0.0012 - val_loss: 7.8286 - val_accuracy: 0.0000e+00
Epoch 2/200
13/13 [==============================] - 4s 282ms/step - loss: 7.8443 - accuracy: 0.0012 - val_loss: 7.8248 - val_accuracy: 0.0000e+00
Epoch 3/200
13/13 [==============================] - 4s 279ms/step - loss: 7.8182 - accuracy: 0.0012 - val_loss: 7.8205 - val_accuracy: 0.0000e+00
Epoch 4/200
13/13 [==============================] - 4s 279ms/step - loss: 7.7891 - accuracy: 0.0025 - val_loss: 7.8155 - val_accuracy: 0.0000e+00
Epoch 5/200
13/13 [==============================] - 4s 307ms/step - loss: 7.7637 - accuracy: 0.0074 - val_loss: 7.8098 - val_accuracy: 0.0000e+00
Epoch 6/200
13/13 [==============================] - 4s 281ms/step - loss: 7.7378 - accuracy: 0.0123 - val_loss: 7.8034 - val_accuracy: 0.0000e+00
Epoch 7/200
13/13 [==============================] - 4s 280ms/step - loss: 7.7085 - accuracy: 0.0198 - val_loss: 7.796

In [15]:
index_list = []
for i in range(1,len(pred_label_list)+1):
    index_list.append(f"S{i:03}")

#prediction = pd.DataFrame(pred_label_list, index = index_list)

prediction = pd.DataFrame(columns=['id', 'pred'])

prediction["id"] = index_list
prediction["pred"] = pred_label_list

prediction = prediction.reset_index(drop=True)

#prediction = pd.DataFrame(pred_label_list, index=[f"S{i:03}" for i in range(1, len(pred_label_list)+1)])

#prediction.columns = ["id","pred"]
#prediction.index = index_list
prediction.to_csv('20221119_하준서_simple_seq.answer.csv', index = False)

#index_list
prediction

,id,pred
0,S001,D20
1,S002,D16
2,S003,D3
3,S004,D16
4,S005,D16
...,...,...
95,S096,D20
96,S097,D12
97,S098,D20
98,S099,D12
